# Data collection and storing with SQLite
Data downloaded from:
[http://data.dft.gov.uk/anonymised-mot-test/test_data/dft_test_result_2019.zip](https://data.gov.uk/dataset/e3939ef8-30c7-4ca8-9c7c-ad9475cc9b2f/anonymised-mot-tests-and-results)

Zip file extracted to local disk, then 4 csv files put into the `mot-data/data` folder

In [1]:
# Import and create a new database file
import sqlite3
from sqlite3 import Error
import os

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

connection = create_connection(os.path.join('data','mot.sqlite'))

Connection to SQLite DB successful


In [2]:
# Create function to simply execute query
def execute_query(connection, query, firstonly=False, verbose=True):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        if verbose:print("Query executed successfully");
    except Error as e:
        print(f"The error '{e}' occurred")
    if firstonly:
        return cursor.fetchone()
    else:
        return cursor.fetchall()

def insert_query(connection, query, t, verbose=True):
    cursor = connection.cursor()
    try:
        cursor.executemany(query, t)
        connection.commit()
        if verbose:print("Query executed successfully");
    except Error as e:
        print(f"The error '{e}' occurred")
    return

In [3]:
import pandas as pd
import os

names = ['test_id','vehicle_id','test_date','test_class_id','test_type','test_result','test_mileage','postcode_area','make','model','colour','fuel_type','cylinder_capacity','first_use_date']
dtypes = {'test_id':'int', 
          'vehicle_id':'int',
          'test_date':'object', 
          'test_class_id':'int',
          'test_type':'object', 
          'test_result':'object', 
          'test_mileage':pd.Int64Dtype(),
          'postcode_area':'object', 
          'make':'object', 
          'model':'object', 
          'colour':'object',
          'fuel_type':'object', 
          'cylinder_capacity':'object', 
          'first_use_date':'object'}
na_values = {'test_mileage':-1}

In [32]:
# Create mot1 table
execute_query(connection,'''
DROP TABLE IF EXISTS mot1;
''')

execute_query(connection,'''
DROP TABLE IF EXISTS passtable;
''')

execute_query(connection, '''
CREATE TABLE mot1 (
  test_id INTEGER,
  vehicle_id INTEGER,
  test_date TEXT,
  test_class_id INTEGER,
  test_type TEXT,
  test_result INTEGER,
  test_mileage INTEGER,
  postcode_area TEXT,
  make TEXT,
  model TEXT,
  colour TEXT,
  fuel_type TEXT,
  cylinder_capacity INTEGER,
  first_use_date TEXT,
  FOREIGN KEY(test_result) REFERENCES passtable(test_result)
);
''')

execute_query(connection, '''
CREATE TABLE passtable (
    test_result INTEGER,
    test_result_code TEXT,
    description TEXT
);
''')

insert_query(connection, '''
INSERT INTO passtable (test_result, test_result_code, description)
VALUES (?,?,?)
''', [(0,'P','Pass'),
     (1,'F','Fail'),
     (2,'PRS','Pass with rectification'),
     (3,'ABA','Abandon with fee charged'),
     (4,'ABR','Abort with no fee due to equipment etc'),
     (5,'ABRVE','Abort with no fee due to examiner')])

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully


In [5]:
print(*execute_query(connection,'SELECT * FROM mot1 limit 5;', verbose=False))

In [6]:
# Verify passtable looks right
execute_query(connection,'SELECT * FROM passtable limit 5;', verbose=False)

[(0, 'P', 'Pass'),
 (1, 'F', 'Fail'),
 (2, 'PRS', 'Pass with rectification'),
 (3, 'ABA', 'Abandon with fee charged'),
 (4, 'ABR', 'Abort with no fee due to equipment etc')]

In [30]:
print(os.path.isfile(os.path.join('data','dft_test_result-from-2019-04-01_00-00-01-to-2019-07-01_00-00-01.csv')))
[x for x in os.listdir('data') if '.csv' in x]
# os.path.isfile(f'data/{a[0]}')
# print(a)

True


['dft_test_result-from-2019-04-01_00-00-01-to-2019-07-01_00-00-01.csv',
 'dft_test_result-from-2019-01-01_00-00-01-to-2019-04-01_00-00-01.csv',
 'dft_test_result-from-2019-10-01_00-00-01-to-2020-01-01_00-00-01.csv',
 'dft_test_result-from-2019-07-01_00-00-01-to-2019-10-01_00-00-01.csv']

In [33]:
# Modidfied copy
from datetime import datetime
list_nats = []

def mot_to_sql(path, connection, tablename, list_nats=[], if_exists='append'):
    # Alternative simpler code, straight from dataframe to sql.
    for i, chunk in enumerate(pd.read_csv(path,
                             sep=',', delim_whitespace=False,
                             names=names, dtype=dtypes, engine='python',
                             chunksize=500000, iterator=True, header=0,
                             error_bad_lines=False, warn_bad_lines=True, doublequote=False)):
        #DEBUGGING
        #if i>10:break

        # Encode pass results to adhere to lookup table
        chunk.loc[chunk['test_result'] == 'P','test_result'] = 0
        chunk.loc[chunk['test_result'] == 'F','test_result'] = 1
        chunk.loc[chunk['test_result'] == 'PRS','test_result'] = 2
        chunk.loc[chunk['test_result'] == 'ABA','test_result'] = 3
        chunk.loc[chunk['test_result'] == 'ABR','test_result'] = 4
        chunk.loc[chunk['test_result'] == 'ABRVE','test_result'] = 5
        chunk['test_result'] = chunk['test_result'].astype(int)

        # Clean nan mileages
        chunk.loc[pd.isna(chunk['test_mileage']),'test_mileage'] = -1

        # Clean nan engine sizes
        chunk.loc[pd.isna(chunk['cylinder_capacity']),'cylinder_capacity'] = -1

        # Clean dates
        chunk['test_date'] = pd.to_datetime(chunk['test_date'],format='%Y-%m-%d', errors='coerce')
        chunk['first_use_date'] = pd.to_datetime(chunk['first_use_date'],format='%Y-%m-%d', errors='coerce')

        chunk.to_sql(tablename, connection, if_exists=if_exists, index = False)
        print(f'chunknum {i} completed.')
        
        is_nat = pd.isna(chunk['first_use_date'])
        if any(is_nat):
            list_nats.extend(chunk[is_nat]['test_id'].values)
            print('...NaT detected')

    return list_nats

list_nats = mot_to_sql(os.path.join('data','dft_test_result-from-2019-04-01_00-00-01-to-2019-07-01_00-00-01.csv'),connection,'mot1',list_nats=list_nats,if_exists='append')
print('File1 nats: ',list_nats)
list_nats = mot_to_sql(os.path.join('data','dft_test_result-from-2019-01-01_00-00-01-to-2019-04-01_00-00-01.csv'),connection,'mot1',list_nats=list_nats,if_exists='append')
print('File2 nats: ',list_nats)
list_nats = mot_to_sql(os.path.join('data','dft_test_result-from-2019-10-01_00-00-01-to-2020-01-01_00-00-01.csv'),connection,'mot1',list_nats=list_nats,if_exists='append')
print('File3 nats: ',list_nats)
list_nats = mot_to_sql(os.path.join('data','dft_test_result-from-2019-07-01_00-00-01-to-2019-10-01_00-00-01.csv'),connection,'mot1',list_nats=list_nats,if_exists='append')
print('File4 nats: ',list_nats)


chunknum 0 completed.
...NaT detected
chunknum 1 completed.
chunknum 2 completed.
chunknum 3 completed.
chunknum 4 completed.
chunknum 5 completed.
chunknum 6 completed.
chunknum 7 completed.
...NaT detected
chunknum 8 completed.
...NaT detected
chunknum 9 completed.
chunknum 10 completed.
chunknum 11 completed.
...NaT detected
chunknum 12 completed.
chunknum 13 completed.
chunknum 14 completed.
chunknum 15 completed.
...NaT detected
chunknum 16 completed.
...NaT detected
chunknum 17 completed.
chunknum 18 completed.
...NaT detected
chunknum 19 completed.
chunknum 20 completed.
...NaT detected
File1 nats:  [1413867915, 494067041, 458084789, 1572247029, 1480912787, 1706775143, 1519590599, 549522641, 275261921, 68409671]
chunknum 0 completed.
chunknum 1 completed.
...NaT detected
chunknum 2 completed.
chunknum 3 completed.
chunknum 4 completed.
...NaT detected
chunknum 5 completed.
chunknum 6 completed.
...NaT detected
chunknum 7 completed.
...NaT detected
chunknum 8 completed.
...NaT de

# Test the database

In [ ]:
df = pd.read_sql_query('''SELECT * FROM mot1 
LEFT JOIN passtable 
ON passtable.test_result = mot1.test_result;''', connection)

import sys
print(sys.getsizeof(df),'bytes')
df.info()

In [37]:
# Show schema
print(execute_query(connection,'''
SELECT sql
FROM sqlite_master
WHERE type="table"
AND name="mot1";
''')[0][0])

Query executed successfully
CREATE TABLE mot1 (
  test_id INTEGER,
  vehicle_id INTEGER,
  test_date TEXT,
  test_class_id INTEGER,
  test_type TEXT,
  test_result INTEGER,
  test_mileage INTEGER,
  postcode_area TEXT,
  make TEXT,
  model TEXT,
  colour TEXT,
  fuel_type TEXT,
  cylinder_capacity INTEGER,
  first_use_date TEXT,
  FOREIGN KEY(test_result) REFERENCES passtable(test_result)
)


In [36]:
print(execute_query(connection,'''
SELECT COUNT(test_id)
FROM mot1
'''))

Query executed successfully
[(39310698,)]


In [34]:
list_nats

[1413867915,
 494067041,
 458084789,
 1572247029,
 1480912787,
 1706775143,
 1519590599,
 549522641,
 275261921,
 68409671,
 1508254279,
 1487206527,
 1197994457,
 1371175063,
 1969863607,
 713113865,
 1926654095,
 758782857,
 20432289,
 1951363753,
 267869835,
 151909805,
 1857516703,
 547246697,
 594809191,
 602339371,
 312054029,
 1580590695,
 1467572141,
 245493977,
 1546096289,
 1212904601,
 1788288741,
 1203829975,
 1417027143,
 1587079331,
 606207757,
 152895981,
 1516762995]

In [35]:
# Save to disk a copy of list_nats for future reference
import pickle
pickle.dump(list_nats,open( "list_nats.pkl", "wb" ))

**Code for my debugging**

To read specific line from csv file without loading all into memory

In [ ]:
# Read specific line content from large csv file based on line number
lines = list(range(7795035, 7795036))

with open(os.path.join('data','dft_test_result-from-2019-01-01_00-00-01-to-2019-04-01_00-00-01.csv')) as fp:
    for i, line in enumerate(fp):
        if i in lines:
            print(f'[{i}]{repr(line)}')
        elif i > max(lines):
            break